# Celestial Coordinates with WCS 1: Specifying, reading and plotting

## Authors

Kris Stern, Kelle Cruz, Lia Corrales, David Shupe

## Learning Goals

1. Two ways to build one's own astropy.wcs.WCS object
2. Show an image of the Helix nebula with RA and Dec labeled
3. Plot a scale bar on an image with WCS

## Keywords

WCS, coordinates, matplotlib

## Companion Contents
1. "An Introduction to Modern Astrophysics" ([Carroll & Ostlie](https://ui.adsabs.harvard.edu/abs/2006ima..book.....C/abstract))
2. [FITS WCS page at GSFC](https://fits.gsfc.nasa.gov/fits_wcs.html)

## Summary

This tutorial series aims to show how the content of Chapter 1 of "An Introduction to Modern Astrophysics" by Carroll and Ostlie can be applied to real life astrohpysics research situations, using tools in the Astropy ecosystem. We will introduce two different approaches to building one's own `astropy.wcs.WCS` object, which conforms to the standards of the FITS World Coordinate System (WCS) used extensively by the astronomy research community. We will createda 2D WCS for an image of the iconic the Helix nebula (a planetary nebula) and plot an image of the nebula with RA and DEC labeled. Finally, we will overplot a scale bar on the Helix nebula image with WCS. 

In [ ]:
from astropy.wcs import WCS
import requests
from astropy.io import fits
import matplotlib.pyplot as plt

## Section 1: Two ways to build your own `astropy.wcs.WCS` object

*World coordinates* serve to locate a measurement in some multi-dimensional parameter space. A World Coordinate System (WCS) specifies the physical, or world, coordintes to be attached to each pixel or voxel of an N-dimensional image or array. An [elaborate set of standards and conventions](https://fits.gsfc.nasa.gov/fits_wcs.html) have been developed for the Flexible Image Transport System (FITS) format ([Wells et al. 1981](https://ui.adsabs.harvard.edu/abs/1981A&AS...44..363W/abstract)). A typical WCS example is to specify the Right Ascension (RA) and Declination (Dec) on the sky associated with a given the pixel or spaxel location in a 2-dimensional celestial image ([Greisen & Calabretta 2002](https://ui.adsabs.harvard.edu/abs/2002A&A...395.1061G/abstract); [Calabretta and Greisen 2002](https://ui.adsabs.harvard.edu/abs/2002A&A...395.1077C/abstract)).

The [`astropy.wcs` subpackage](https://docs.astropy.org/en/stable/wcs/) implements FITS standards and conventions for World Coordinate Systems. Using the `astropy.wcs.WCS` object and `matplotlib`, we can generate images of the sky that have axes labeled with coordinates such as right ascension (RA) and declination (Dec). This requires selecting the proper projections for `matplotlib` and providing a `WCSAxes` object.

**There are two main ways to initialize a `WCSAxes` object: with a Python dictionary or with Python lists.** In this set of examples, we will initialize an `astropy.wcs.WCS` object with two axes. The axes are defined by `CTYPE` keywords. 

Here is a list of essential WCS keywords and their uses. In each case, the integer $n$ denotes the dimensional axis (starting with 1) to which the keyword is being applied.

* **CRVALn**: the coordinate value at the reference point (e.g., RA and DEC value in degrees)
* **CRPIXn**: the pixel location for the reference point (e.g., CRPIX1=1, CRPIX2=1 describes the center of a corner pixel)
* **CDELTn**: the coordinate increment at the reference point (e.g., the difference in 'RA' value from the reference pixel to its neighbor along the RA axis)
* **CTYPEn**: an 8-character string describing the axis type (e.g., 'RA---TAN' and 'DEC---TAN' describe typical sky projections that astronomers use)
* **CUNITn**: a string describing the unit for each axis (If not specified, the default unit is degrees.)
* **NAXISn** : an integer defining the number of pixels in each axis

Some good references of the WCS can be found [here](https://fits.gsfc.nasa.gov/fits_wcs.html).

### Method 1: Building a WCS with a dictionary

One way to define an Astropy WCS object is to construct a dictionary containing all the essential information mapping the pixel coordinate space to the world coordinate space. The advantage of this approach is that the `NAXIS` value for each axis can be included directly. 

In this example, we define a two coordinate axes with:
* A Gnomic (tangent-plane) projection, which corresponds to the RA/Dec coordinate system
* A reference location of (RA,DEC) = (337.52, -20.83), as defined by the **CRVALn** keys
* The pixel at coordinate value (1,1) as the reference location (**CRPIXn** keys)
* Units of degrees (**CUNITn = 'deg'**)
* Pixel sizes of 1 x 1 arcsec (**CDELTn = 0.002778** in degrees)
* An image size of 1024 x 1024 pixels (**NAXISn** key)

In [ ]:
wcs_input_dict = {
'CTYPE1': 'RA---TAN', 'CUNIT1': 'deg', 'CDELT1': -0.0002777777778, 'CRPIX1': 1, 'CRVAL1': 337.5202808, 'NAXIS1': 1024,
'CTYPE2': 'DEC--TAN', 'CUNIT2': 'deg', 'CDELT2': 0.0002777777778, 'CRPIX2': 1, 'CRVAL2': -20.833333059999998, 'NAXIS2': 1024}
wcs_helix_dict = WCS(wcs_input_dict)

Now let's print the `WCS` object defined with a Python dictionary to check its content: 

In [ ]:
wcs_helix_dict # To check output

### Method 2: Create an empty WCS object before assigning values

Alternatively, we could initialize the `astropy.wcs.WCS` object, and assign the keyword values with lists corresponding to each respective axis.

In [ ]:
wcs_helix_list = WCS(naxis=2)
wcs_helix_list.wcs.crpix = [1, 1]
wcs_helix_list.wcs.crval = [337.5202808, -20.833333059999998]
wcs_helix_list.wcs.cunit = ["deg", "deg"]
wcs_helix_list.wcs.ctype = ["RA---TAN", "DEC--TAN"]
wcs_helix_list.wcs.cdelt = [-0.0002777777778, 0.0002777777778]

Let's print the `WCS` object one more time to check how our values were assigned.

In [ ]:
wcs_helix_list # To check output

Note that when we initialize the WCS object this way, the `NAXIS` values are set to 0. To assign coordinates to our image, we will need to fix the shape of the `WCS` object array so that it matches our image. We can do this by assigning a value to the `array_shape` attribute of the `WCS` object:

In [ ]:
wcs_helix_list.array_shape = [1024, 1024]

Now when we print the `WCS` object, we can see that the `NAXIS` values have been updated from the default size of 0 to 1024.

In [ ]:
wcs_helix_list

## Section 2: Show an image of the Helix nebula with RA and Dec labeled

Most of the time we can obtain the required `astropy.wcs.WCS` object from the header of the FITS file from a telescope or astronomical database. This process is described below.

### Step 1: Read in the FITS file

First, we will download the FITS file from Zenodo using the Python requests form. Then we will read the FITS file into a variable called header_data_unit.

In [ ]:
header_data_unit_list = fits.open('https://github.com/astropy/astropy-data/raw/6d92878d18e970ce6497b70a9253f65c925978bf/tutorials/celestial-coords1/tailored_dss.22.29.38.50-20.50.13_60arcmin.fits')

FITS files are formatted to be used by astronomers and can contain information arranged in many "extensions." We can check how many extensions there are in a FITS file, as well as view a summary of the contents in each extension, by printing the FITS object information.

In [ ]:
header_data_unit_list.info()

This file contains only one extension, labeled 'PRIMARY' (or extension number 0). We will copy the image data to the variable `image`, and the header data to the variable `header`.

In [ ]:
image = header_data_unit_list[0].data
header = header_data_unit_list[0].header

We can print the FITS image header to screen so that all of its contents can be checked or utilized. Note that the WCS information for this information can be found near the bottom of the printed header, below.

In [ ]:
header

Please note that the original header violates the FITS WCS standards (including both CDELTn and CD-matrix; including deprecated PC-matrix keywords). The header has been cleaned up to conform to the existing standards.

### Step 2: Read in the FITS image coordinate system with astropy.wcs.WCS

Because the header contains WCS information, the Astropy `WCS` class can be loaded directly from the FITS header information.

In [ ]:
wcs_helix = WCS(header)

Let's print the `WCS` object to see what values were drawn from the header.

In [ ]:
wcs_helix

### Step 3: Plot the Helix nebula with the proper RA and Dec

Next, let's view the image. We will be using the `matplotlib` library for the plotting. 

The `WCS` object is built so it can be provided to `matplotlib` with the  `projection` keyword, as shown in the call to `matplotlib.pyplot.subplot`, below.

In [ ]:
fig = plt.figure(figsize=(10,10), frameon=False)
ax = plt.subplot(projection=wcs_helix)
plt.imshow(image, origin='lower', cmap='cividis', aspect='equal')
plt.xlabel(r'$RA$')
plt.ylabel(r'$Dec$')
plt.show()

## Exercise

What happens if we do not use the WCS projection? Re-run the code block above without defining the subplot projection as `wcs_helix`. 

In [ ]:
fig = plt.figure(figsize=(10,10), frameon=False)
ax = plt.subplot(111)
plt.imshow(image, origin='lower', cmap='cividis', aspect='equal')
plt.xlabel(r'$Physical \; X$')
plt.ylabel(r'$Physical \; Y$')
plt.show()

Compare your two plots, with and without the wcs projection. In what cases would you prefer to see the image with RA and Dec? In what cases are the pixel coordinates more useful?

## Section 3: Plot a scale bar on an image with WCS

To add a scale bar to the image of the Helix nebula, we will use the matplotlib `Axes.arrow` method to draw a line. 

First, we need to decide where to place the scale bar. In the example below, the center of the scale bar is at `(RA, Dec) = (337 deg, -21.2 deg)`. 

Then, we can use the `transform` attribute of `Axes.arrow` to draw our scale bars in degrees. In this case, we draw a scale bar with a length of 0.1 degrees. The arrow method inputs are `ax.arrow(x, y, dx, dy, **kwargs)`, with `x` being the `RA` and `y` being the `Dec` of for one end of the arrow. We use `dx=0` so that there is no horizontal component in the bar, and `dy=0.1` gives the length of the arrow in the vertical direction. To ensure that the arrow is drawn in the J2000 FK5 coordinate frame, we pass `ax.get_transform('fk5')` to the `transform` keyword.

Finally, we use `matplotlib.pyplot.text` to describe the length of the scale bar.

In [ ]:
fig = plt.figure(figsize=(10,10), frameon=False)
ax = plt.subplot(projection=wcs_helix)
ax.arrow(337, -21.2, 0, 0.1, head_width=0, head_length=0, fc='white', ec='white', width=0.003, transform=ax.get_transform('fk5'))
plt.text(337.05, -21.18, '0.1 deg', color='white', rotation=90, transform=ax.get_transform('fk5'))
plt.imshow(image, origin='lower', cmap='cividis', aspect='equal')
plt.xlabel(r'$RA$')
plt.ylabel(r'$Dec$')
plt.show()

## Exercise

Make a horizontal bar with the same length. Keep in mind that 1 hour angle = 15 degrees.